# Open Loop Translation Control

## Step 1: Write a Python Script

Your first task in this module is to write a Python script that uses setpoint commands to make your UAV fly a square pattern.

## Step 2: Setup for Test Flight:

Follow these steps to prepare for a flight test:

- Power on remote control
- Power on the Intel RTF
- Connect to the UAV via QGroundControl (need to know IP address from previous flights or connect a monitor and run ifconfig)
- SSH into the UAV (you will need to have at least four terminals SSHed into the UAV)
- In one terminal, start the teraranger and optical flow services:
    
    ~~~~
    sudo systemctl start aero-teraranger.service
    sudo systemctl start aero-optical-flow
    ~~~~
    
<div class="alert alert-info">
**Note:** before you move on, go into QGroundControl and make sure the teraranger and optical flow are working properly. See [Verify Services](https://github.mit.edu/pages/BWSI/UAVRacing/enable_optical_flow.html#Step-3:-Verify-Services)
</div>
    
- In the next terminal, start the ROS Master:

    ~~~~
    roscore
    ~~~~
- In a third terminal, start the mavros node:

    ~~~~
    rosrun mavros mavros_node _fcu_url:=tcp://127.0.0.1:5760 _system_id:=2
    ~~~~

- In a fourth terminal, `cd` into the catkin package the contains your open loop translation script. Run the script:

    ~~~~
    python <your_script>
    ~~~~
    
## Step 3: Test Flight

Once your code is running, you are now ready to fly. 
- Manually arm the motors and take off
- Get the UAV stable in the air (it is highly recommend to switch into position mode after takeoff)
- Once the UAV is stable, switch into offboard mode. This should prompt the UAV to fly its programmed course.
- After you UAV has performed its flight path, manually retake control of the drone and land.
